solutions for puzzles from the Riddler, found at https://fivethirtyeight.com/features/can-you-win-a-spelling-bee-if-you-know-99-percent-of-the-words/

### Riddler Express 

   This question is actually fairly easy to intuit. At the end of the first game, you have seen 6 out of the 30 cards. What the question is then asking is what are the odds you can draw three <u>more</u> cards that you haven't essentially picking 3 out of the pool of 24 as-yet-unseen cards. This can be easily shown by working out the odds of drawing each card sequentially without replacement. The odds of the first would be 24/30, the second 23/29, and the third 22/28.

In [1]:
(24 * 23 * 22) / (30 * 29 * 28)

0.49852216748768474

The logic behind that is a little more complicated, but not much. The most important this case is the number of subsets r given a population N, which is the formula (N!)/r!(N-r)!, or the binomial coefficient.

Using this formula, we can find 1) the number of ways we can pick the 3 cards we haven't seen, and 2) the total number of possible ways to pick 3 cards out of 30. Since P(e) is N(e)/N <i>(probability of some event is the number of times that outcome occurs divided by the total number of all outcomes)</i>, we can then divide the first number by the second to get our answer.

In [2]:
from functools import reduce

def factorial(n):
    return reduce(lambda x, y: x * y, range(1, n+1)) # does not work for 0, not needed for this problem
    # also could have imported math.factorial but any practice with lambdas is good practice

def bin_coeff(N, r):
    num = factorial(N) # numerator
    denom = factorial(r) * factorial(N - r) # denominator
    return num/denom

In [3]:
bin_coeff(24, 3) / bin_coeff(30, 3)

0.49852216748768474

### Riddler Classic

This problem is decently more complicated... as I don't know the math procedure to easily calculate this by hand, then we will brute force it with programming instead!

First, lets define a function that will return true if the word is within the knowledge of the speller, and false otherwise

In [4]:
from random import randint

def f(n):
    return randint(1,100) < n

Here is the meat of the simulation... we provide the number of trials wanted and the order of the spellers for each trial. Then, we create a zeroes array to collect the winnrs of each trial. During each trial, we create a list of spellers in the specified order. Each speller is identified by the percentage of the dictionary they have memorized. While the number of spellers is 2 or more, cycle through the list. If the speller gets the word right (the function returns true) continue, if they get it false remove the speller and continue the rotation. Do this until there is only one speller remaining. At this point, increment the results by that spellers percentage of knowledge mod 90 (0 through 9 from weakest to strongest speller), then continue with the next cycle. Once done, return the final results array.

In [5]:
def run_trials(num_trials, desc_order):
    results = [0] * 10
    for _ in range(num_trials):
        if desc_order:
            spellers = list(range(99,89,-1))
        else:
            spellers = list(range(90,100))
        i = 0

        while len(spellers) > 1:
            if not f(spellers[i % len(spellers)]):
                del spellers[i % len(spellers)]
            i += 1

        results[spellers[0] % 90] += 1
        
    return results

In [6]:
# 5 runs of 10000 trials of highest-to-lowest order
for _ in range(5):
    print(run_trials(10000, True))

[100, 170, 216, 275, 378, 585, 913, 1304, 2142, 3917]
[112, 140, 210, 297, 384, 594, 812, 1354, 2195, 3902]
[117, 150, 218, 304, 389, 551, 785, 1386, 2197, 3903]
[116, 154, 178, 277, 351, 554, 809, 1351, 2214, 3996]
[110, 137, 197, 283, 377, 562, 824, 1300, 2163, 4047]


In [7]:
# 5 runs of 10000 trials of lowest-to-highest order

In [8]:
for _ in range(5):
    print(run_trials(10000, False))

[101, 119, 198, 259, 422, 568, 864, 1418, 2156, 3895]
[76, 121, 212, 242, 383, 605, 864, 1355, 2244, 3898]
[97, 114, 203, 278, 422, 597, 905, 1353, 2213, 3818]
[76, 108, 175, 310, 400, 562, 829, 1403, 2248, 3889]
[98, 139, 188, 233, 404, 607, 891, 1395, 2247, 3798]


Curiously, it actually appears that the stronger speller has the greatest chance of winning if they go first, as opposed to going last! The other obvious difference is that the weakest speller has a much better chance of winning if going last instead of going first. This conclusion makes sense (the weak speller has a greater chance of other, stronger spellers being disqualified before he goes), but I have no idea why going last would be to the detriment of the strongest speller. Interesting...